In [1]:
import random
import numpy as np
import sys
from random import randint
from data_load import *
in_dir= "/Users/mayk/working/figer/baseline/PLE/Intermediate/BBN"
a=MentionData('/Users/mayk/working/figer/baseline/PLE/Intermediate/BBN/train_x_new.txt',
              "/Users/mayk/working/figer/baseline/PLE/Intermediate/BBN/train_y.txt",
             in_dir+"/feature.txt",in_dir+"/type.txt")
%load_ext Cython

In [2]:
%%cython
import numpy as np

cimport numpy as np
from random import randint
import sys
import cython

import math
from libc.stdlib cimport malloc, free

from libc.math cimport exp
from libc.math cimport log
from libc.math cimport sqrt
from libc.string cimport memset
import random
# scipy <= 0.15

import scipy.linalg.blas as fblas
ctypedef np.float32_t REAL_t
cdef int ONE = 1


REAL = np.float32
cdef extern from "/Users/mayk/working/figer/baseline/PLE/Model/warp/voidptr.h":
    void* PyCObject_AsVoidPtr(object obj)
DEF MAX_SENTENCE_LEN = 10000
ctypedef void (*scopy_ptr) (const int *N, const float *X, const int *incX, float *Y, const int *incY) nogil
ctypedef void (*saxpy_ptr) (const int *N, const float *alpha, const float *X, const int *incX, float *Y, const int *incY) nogil
ctypedef float (*sdot_ptr) (const int *N, const float *X, const int *incX, const float *Y, const int *incY) nogil
ctypedef double (*dsdot_ptr) (const int *N, const float *X, const int *incX, const float *Y, const int *incY) nogil
ctypedef double (*snrm2_ptr) (const int *N, const float *X, const int *incX) nogil
ctypedef void (*sscal_ptr) (const int *N, const float *alpha, const float *X, const int *incX) nogil


cdef scopy_ptr scopy = <scopy_ptr>PyCObject_AsVoidPtr(fblas.scopy._cpointer)  # y = x
cdef saxpy_ptr saxpy=<saxpy_ptr>PyCObject_AsVoidPtr(fblas.saxpy._cpointer)  # y += alpha * x
cdef sdot_ptr sdot=<sdot_ptr>PyCObject_AsVoidPtr(fblas.sdot._cpointer)  # float = dot(x, y)
cdef dsdot_ptr dsdot=<dsdot_ptr>PyCObject_AsVoidPtr(fblas.sdot._cpointer)  # double = dot(x, y)
cdef snrm2_ptr snrm2=<snrm2_ptr>PyCObject_AsVoidPtr(fblas.snrm2._cpointer)  # sqrt(x^2)
cdef sscal_ptr sscal=<sscal_ptr>PyCObject_AsVoidPtr(fblas.sscal._cpointer) # x = alpha * x
DEF EXP_TABLE_SIZE = 10000
DEF MAX_EXP = 50

cdef REAL_t[EXP_TABLE_SIZE] EXP_TABLE
cdef REAL_t[EXP_TABLE_SIZE] LOG_TABLE

cdef REAL_t ONEF = <REAL_t>1.0

# for when fblas.sdot returns a double
cdef REAL_t our_dot_double(const int *N, const float *X, const int *incX, const float *Y, const int *incY) nogil:
    return <REAL_t>dsdot(N, X, incX, Y, incY)

# for when fblas.sdot returns a float
cdef REAL_t our_dot_float(const int *N, const float *X, const int *incX, const float *Y, const int *incY) nogil:
    return <REAL_t>sdot(N, X, incX, Y, incY)

# for when no blas available
cdef REAL_t our_dot_noblas(const int *N, const float *X, const int *incX, const float *Y, const int *incY) nogil:
    # not a true full dot()-implementation: just enough for our cases
    cdef int i
    cdef REAL_t a
    a = <REAL_t>0.0
    for i from 0 <= i < 50 by 1:
        a += X[i] * Y[i]
    return a

# for when no blas available
cdef void our_saxpy_noblas(const int *N, const float *alpha, const float *X, const int *incX, float *Y, const int *incY) nogil:
    cdef int i
    for i from 0 <= i < N[0] by 1:
        Y[i * (incY[0])] = (alpha[0]) * X[i * (incX[0])] + Y[i * (incY[0])]
cdef REAL_t cvdot(vec1,vec2,size):
    cdef int csize = size
    f= dsdot(&csize,<REAL_t *>(np.PyArray_DATA(vec1)),&ONE,<REAL_t *>(np.PyArray_DATA(vec2)),&ONE)
    return f
def csaxpy(vec1,vec2,alpha,size):
    cdef int csize = size
    cdef float calpha = alpha
    f= our_saxpy_noblas(&csize,&calpha,<REAL_t *>(np.PyArray_DATA(vec1)),&ONE,<REAL_t *>(np.PyArray_DATA(vec2)),&ONE)
    return f
cdef REAL_t crank(int k):
    cdef REAL_t loss = 0.
    cdef int i = 1
    for i in xrange(1,k+1):
        loss += ONEF/i
    return loss
cdef REAL_t vsum(REAL_t *vec,int *size):
    cdef int i
    cdef REAL_t product
    product = <REAL_t>0.0
    for i from 0 <= i < size[0] by 1:
        product += vec[i] * vec[i]
    return np.sqrt(product)
def cnorm(vec):
    cdef int size
    size  = len(vec)
    return vsum(<REAL_t *>(np.PyArray_DATA(vec)),&size)
def init():
    for i in range(EXP_TABLE_SIZE):
        EXP_TABLE[i] = <REAL_t>exp((i / <REAL_t>EXP_TABLE_SIZE * 2 - 1) * MAX_EXP)
        EXP_TABLE[i] = <REAL_t>(EXP_TABLE[i] / (EXP_TABLE[i] + 1))
#init()


def ctrain(A,B,insts,size,lr,gradient,max_it =10):
    cdef float error

    for it in xrange(1,max_it+1):
        error = 0.
        for i,inst in enumerate(insts):
            error+=gradient(A,B,inst,size,lr=lr)
        
            if i % 1000 ==0:
                sys.stdout.write("\rIteration %d " % (it)+ "trained {0:.0f}%".format(float(i)*100/len(insts))+" Loss:{0:.2f}".format(error))
                sys.stdout.flush()
        sys.stdout.write("\n")
    return error

cdef void divide(REAL_t *vec, const float *alpha, const int *size):
    cdef int i
    cdef REAL_t product
    for i from 0 <= i < size[0] by 1:
        vec[i] = vec[i]/alpha[0]
def cdivide(vec,alpha):
    cdef int size
    size  = len(vec)
    cdef float r = alpha
    divide(<REAL_t *>(np.PyArray_DATA(vec)),&r,&size)

    
def max_margin_gradient(A,B,inst,size,lr=0.01):
    #print B
    #print B[0]-B[9]

    dA = np.zeros(size,dtype=REAL)
    dB = np.zeros([len(inst.labels),size],dtype=REAL)
    random.seed(1)
    x = np.sum(A[inst.features],axis=0)
    cdef REAL_t error = 0
    cdef REAL_t s1,s2
    cdef clr = lr
    cdef int N
    cdef int n_sample
    cdef int neg_num = len(inst.negative_labels)
    cdef REAL_t norm
    for l in inst.sparse_labels:
        s1= cvdot(x,B[l],50)
        N=1
        n_sample  = -1
        for k in xrange(neg_num):
            nl = inst.negative_labels[randint(0,neg_num-1)]
            s2 = cvdot(x,B[nl],50)
            if s1 - s2<1:
                n_sample = nl
                N = k+1
                break
        if n_sample!=-1:
            error += 1 + s2-s1
            csaxpy(B[l]-B[n_sample],dA,1.0,50)
            csaxpy(x,dB[l],1.0,50)
            csaxpy(x,dB[n_sample],-1.0,50)

    for f in inst.features:
        csaxpy(dA,A[f],clr,50)
        norm = cnorm(A[f])
        if norm >1:
            cdivide(A[f],norm)

    for i in xrange(len(B)):
        csaxpy(dB[i],B[i],clr,50)
        #B[i] += lr*dB[i]
        norm =  cnorm(B[i])
        if norm >1:
            cdivide(B[i],norm)
            #B[i] /=norm
    return error

def max_max_margin_gradient(A,B,inst,size,lr=0.01):
    #print B
    #print B[0]-B[9]

    dA = np.zeros(size,dtype=REAL)
    dB = np.zeros([len(inst.labels),size],dtype=REAL)
    random.seed(1)
    x = np.sum(A[inst.features],axis=0)
    cdef REAL_t error = 0
    cdef REAL_t s1,s2
    cdef clr = lr
    cdef int N
    cdef int neg_num = len(inst.negative_labels)
    cdef REAL_t norm
    cdef int max_l,max_nl
    cdef REAL_t max_s1= float('-inf'),max_s2=float('-inf')
    for l in inst.sparse_labels:
        s1= cvdot(x,B[l],50)
        if s1>=max_s1:
            max_l = l
            max_s1 = s1
    for nl in inst.negative_labels:
        s2= cvdot(x,B[nl],50)
        if s2>=max_s2:
            max_nl = nl
            max_s2 = s2
    if max_s1-max_s2<1:
        error += 1 + max_s2-max_s1
        csaxpy(B[max_l]-B[max_nl],dA,1.0,50)
        csaxpy(x,dB[max_l],1.0,50)
        csaxpy(x,dB[max_nl],-1.0,50)

    for f in inst.features:
        csaxpy(dA,A[f],clr,50)
        norm = cnorm(A[f])
        if norm >1:
            cdivide(A[f],norm)

    for i in xrange(len(B)):
        csaxpy(dB[i],B[i],clr,50)
        #B[i] += lr*dB[i]
        norm =  cnorm(B[i])
        if norm >1:
            cdivide(B[i],norm)
            #B[i] /=norm
    return error
def softmax_gradient(A,B,inst,size,lr=0.01):
    #print B
    #print B[0]-B[9]

    dA = np.zeros(size,dtype=REAL)
    random.seed(1)
    x = np.mean(A[inst.features],axis=0)
    cdef REAL_t error = 0
    cdef REAL_t s1,s2,logs1,logs2,g
    cdef clr = lr
    cdef int neg_num = len(inst.negative_labels)
    cdef REAL_t norm
    cdef REAL_t pos = ONEF
    cdef int csize =size
    for l in inst.sparse_labels:
        logs1= cvdot(x,B[l],50)
        if  logs1 <= -MAX_EXP or  logs1 >= MAX_EXP:
            continue
        s1 = EXP_TABLE[<int>((logs1 + MAX_EXP) * (EXP_TABLE_SIZE / MAX_EXP / 2))]
        error -=logs1
        g = (pos - s1) * clr
        csaxpy(B[l],dA,g,50)
        saxpy(&csize, &g, <REAL_t *>(np.PyArray_DATA(x)), &ONE, <REAL_t *>(np.PyArray_DATA(B[l])), &ONE)
        for k in xrange(neg_num):
            nl = inst.negative_labels[randint(0,neg_num-1)]
            logs2 = cvdot(x,B[nl],50)
            if  logs2 <= -MAX_EXP or  logs2 >= MAX_EXP:
                continue
            s2 = EXP_TABLE[<int>((logs2 + MAX_EXP) * (EXP_TABLE_SIZE / MAX_EXP / 2))]
            if s2 > s1:
                g =  (-s2) * clr
                csaxpy(B[nl],dA,g,50)
                saxpy(&csize, &g, <REAL_t *>(np.PyArray_DATA(x)), &ONE, <REAL_t *>(np.PyArray_DATA(B[nl])), &ONE)
                error += logs2
    

    for f in inst.features:
        csaxpy(dA,A[f],clr,50)
        norm = cnorm(A[f])
        #if norm >1:
         #   cdivide(A[f],norm)

    for i in xrange(len(B)):
        norm =  cnorm(B[i])
       # if norm >1:
        #    cdivide(B[i],norm)
            #B[i] /=norm
    return error

def warp_gradient(A,B,inst,size,lr=0.01):
    #print B
    #print B[0]-B[9]
    dA = np.zeros(size,dtype=REAL)
    dB = np.zeros([len(inst.labels),size],dtype=REAL)
    random.seed(1)
    x = np.sum(A[inst.features],axis=0)
    cdef REAL_t error = 0.

    cdef REAL_t clr = lr
    cdef int N,n_sample 
    cdef int neg_num = len(inst.negative_labels)
    cdef REAL_t norm
    cdef int cSize = size
    cdef REAL_t floats
    
   
    for l in inst.sparse_labels:
        s1= cvdot(x,B[l],50)
        N=1
        n_sample  = -1
        for k in xrange(neg_num):
            nl = inst.negative_labels[randint(0,neg_num-1)]
            s2 = cvdot(x,B[nl],50)
            
            if s1 - s2<1:
                n_sample = nl
                N = k+1
                break
        if n_sample!=-1:

            L = crank(len(inst.negative_labels)/N)
            negL = -L
            error += (1+s2-s1)*L

            csaxpy(B[l]-B[n_sample],dA,L,50)
            
            csaxpy(x,dB[l],L,50)
            csaxpy(x,dB[n_sample],-L,50)
            #print dB[l][0]
    for f in inst.features:
        csaxpy(dA,A[f],clr,50)
        norm = cnorm(A[f])
        if norm >1:
            cdivide(A[f],norm)

    for i in xrange(len(B)):
        csaxpy(dB[i],B[i],clr,50)

        #B[i] += lr*dB[i]
        norm =  cnorm(B[i])
        if norm >1:
            cdivide(B[i],norm)
            #B[i] /=norm
    return error
def save_to_text(matrix,output):
    shape = matrix.shape
    with open(output,'wb') as out:
        out.write("%d %d\n" % (shape))
        for row in matrix:
            x = " ".join(map(lambda x:"{0:.5}".format(x),row))
            out.write(x+"\n")



In [5]:
size=50
np.random.seed(1)
A= np.random.rand(len(a.feature2id),size).astype(np.float32)
# for i in xrange(len(A)):
#     norm =  cnorm(A[i])
#     if norm >1:
#         cdivide(A[i],norm)
B= np.random.rand(len(a.label2id),size).astype(np.float32)
# for i in xrange(len(B)):
#     norm =  cnorm(B[i])
#     if norm >1:
#         cdivide(B[i],norm)
ctrain(A,B,a.data,50,0.01,warp_gradient,max_it=2)

#save_to_text(A,'/Users/mayk/working/figer/baseline/PLE/Results/warp_py_A.txt')
#save_to_text(B,'/Users/mayk/working/figer/baseline/PLE/Results/warp_py_B.txt')

Iteration 1 trained 100% Loss:435254.88
Iteration 2 trained 100% Loss:71137.88


Iteration 1 trained 80% Loss:27027.41
Iteration 2 trained 80% Loss:7855.78


In [ ]:
%%cython
import numpy as np
cimport numpy as np
cdef np.uint32_t *d =<np.uint32_t *>np.PyArray_DATA(np.asarray([[1,2],[1,3]],dtype=np.uint32))
print d[0],d[1],d[2],d[3]

In [4]:
import threading
try:
    from queue import Queue, Empty
except ImportError:
    from Queue import Queue, Empty
import logging
logger = logging.getLogger(__name__)
logging.basicConfig(level=logging.DEBUG)
size=50
np.random.seed(1)
A= np.random.rand(len(a.feature2id),size).astype(np.float32)
# for i in xrange(len(A)):
#     norm =  cnorm(A[i])
#     if norm >1:
#         cdivide(A[i],norm)
B= np.random.rand(len(a.label2id),size).astype(np.float32)
insts = a.data
def train(A,B,insts,it):
    queue_factor = 3
    works = 8
    batch = 2000
    def job_producer():
        """Fill jobs queue using the input `sentences` iterator."""
        job_batch, batch_size = [], 0
        pushed_words, pushed_examples = 0, 0
        job_no = 0

        for inst_idx, inst in enumerate(insts):
            # can we fit this sentence into the existing job batch?
            if batch_size + 1 <= batch:
                # yes => add it to the current job
                job_batch.append(inst)
                batch_size += 1
            else:
                # no => submit the existing job
                logger.debug(
                    "queueing job #%i (%i words, %i sentences)",
                    job_no, batch_size, len(job_batch))
                job_no += 1
                job_queue.put((job_batch))
                # add the sentence that didn't fit as the first item of a new job
                job_batch, batch_size = [inst], 1
        if job_batch:
            logger.debug(
                "queueing job #%i (%i insts, %i insts)",
                job_no, batch_size, len(job_batch))
            job_no += 1
            job_queue.put((job_batch))
        for _ in xrange(works):
            job_queue.put(None)
    def worker_loop():
        jobs_processed = 0
        while True:
            job = job_queue.get()
            if job is None:
                progress_queue.put(None)
                break  # no more jobs => quit this worker
            insts = job
            tally = ctrain(A,B,insts,50,0.01,warp_gradient,max_it=1)
            progress_queue.put((len(insts), tally))  # report back progress
            jobs_processed += 1
    job_queue = Queue(maxsize=queue_factor *works )
    progress_queue = Queue(maxsize=(queue_factor + 1) * works)
    workers = [threading.Thread(target=worker_loop) for _ in xrange(works)]
    unfinished_worker_count = len(workers)
    workers.append(threading.Thread(target=job_producer))
    for thread in workers:
        thread.daemon = True  # make interrupting the process with ctrl+c easier
        thread.start()
    err =0
    job_tally=0
    while unfinished_worker_count > 0:
        report = progress_queue.get()  # blocks if workers too slow
        if report != None:
            err += report[1]
            sys.stdout.write("\rIteration %d " % (it)+ " Loss:{0:.2f}".format(err))
        if report is None:  # a thread reporting that it finished
            unfinished_worker_count -= 1
            logger.info("worker thread finished; awaiting finish of %i more threads", unfinished_worker_count)
            continue
        job_tally += 1
        sys.stdout.flush()
    sys.stdout.write('\n')

In [5]:
%time for i in range(10):train(A,B,insts,i)

Iteration 0  Loss:479154.99
Iteration 1  Loss:83630.64
Iteration 2  Loss:59465.52
Iteration 3  Loss:47082.57
Iteration 4  Loss:40557.20
Iteration 5  Loss:35351.63
Iteration 6  Loss:32013.54
Iteration 7  Loss:28301.94
Iteration 8  Loss:25872.53
Iteration 9  Loss:23327.80
CPU times: user 3min 51s, sys: 38.7 s, total: 4min 30s
Wall time: 4min 2s


In [28]:
def getLabelLinks(fn):
    return processLines(fn,lambda x,y:addFeature(x,y.rstrip().split('\t')),list())
tuples = getLabelLinks('/Users/mayk/working/figer/baseline/PLE/Intermediate/BBN/type_type_kb.txt')
tuples = [ (int(t[0]),int(t[1]),float(t[2]))for t in tuples]
size= 50
def corrReg(B,tup,weight,size):
    diff = B[tup[0]] - B[tup[1]]
    B[tup[0]] -= weight*tup[2]*diff
    B[tup[1]] += weight*tup[2]*diff
    norm = np.linalg.norm(diff)
    return norm * norm * weight*tup[2]

In [43]:
np.random.seed(1)

A= np.random.rand(len(a.feature2id),size).astype(np.float32)
B= np.random.rand(len(a.label2id),size).astype(np.float32)
reg_err =0.
for i in range(10): 
    reg_err =0.
    ctrain(A,B,a.data,50,0.01,warp_gradient,max_it=1)
    for tup in tuples:
        reg_err += corrReg(B,tup,0.2,50)
    print "reg error:",reg_err
save_to_text(A,'/Users/mayk/working/figer/baseline/PLE/Results/warp_py_A.txt')
save_to_text(B,'/Users/mayk/working/figer/baseline/PLE/Results/warp_py_B.txt')

Iteration 1 trained 100% Loss:435254.88
reg error: 1.45011725664
Iteration 1 trained 100% Loss:71077.79
reg error: 1.25802981709
Iteration 1 trained 100% Loss:47323.54
reg error: 1.23192860674
Iteration 1 trained 100% Loss:35948.48
reg error: 1.27711761356
Iteration 1 trained 100% Loss:29882.83
reg error: 1.3362147477
Iteration 1 trained 100% Loss:25117.31
reg error: 1.28632881707
Iteration 1 trained 100% Loss:22028.33
reg error: 1.20925369402
Iteration 1 trained 100% Loss:19977.69
reg error: 1.17426922764
Iteration 1 trained 100% Loss:17786.69
reg error: 1.20700317466
Iteration 1 trained 100% Loss:16225.64
reg error: 1.26068827138


In [ ]:
tuples[0]